In [ ]:
import pandas as pd
import json
from selenium.webdriver.common.keys import Keys
from selenium import webdriver
import gender_guesser.detector as gd
import pycountry
from uszipcode import SearchEngine

C:\Anaconda_env1\lib\site-packages\fuzzywuzzy\fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [ ]:
df = pd.read_csv(r'Final_dataset.xlsx')
coulmns = list(df.columns)
print(coulmns)
data_list=[]

['DATE', 'NAME_TITLE', 'FIRST_NAME', 'LAST_NAME', 'FULL_NAME', 'SSN', 'PHONE', 'EMAIL', 'GENDER', 'STREET_ADDRESS', 'ZIP_CODE', 'CITY', 'STATE', 'COUNTRY', ' PRICE ']


In [ ]:
def get_gender(temp):
    d = gd.Detector()
    out = d.get_gender(u"{}".format(temp))
    if out == "male":
        return "Male"
    elif out == "female":
        return "Female"
    else:
        return "Unknown"

In [ ]:
def generate_email(a):
    b=a.replace(" ","")
    out=b+"@gmail.com"
    return out
#this set of code adds invalid fake entry for email to the person detail which can be excluded for real dataset trials.

In [ ]:
def get_country(state_name):
    try:
        state = pycountry.subdivisions.lookup(state_name)
        country_code = state.country.alpha_2
        out = pycountry.countries.get(alpha_2=country_code).name
        return out
    except LookupError:
        return None

In [ ]:
def get_state(zip_code):
    try:
        search = SearchEngine()
        zipcode = search.by_zipcode(zip_code)
        state_out = zipcode.state
        return state_out
    except Exception:
        return None


In [ ]:
def get_city(zip_code):
    try:
        search = SearchEngine()
        zipcode = search.by_zipcode(zip_code)
        city_out = zipcode.major_city
        return city_out
    except Exception:
        return None

In [ ]:
def get_zipcode(city):
    try:
        search = SearchEngine()
        zipcode = search.by_city(city)
        if zipcode:
            return zipcode[0].zipcode
        else:
            return None
    except Exception:
        return None

In [ ]:
#for i in range(0,len(df)+1):
for i in range(0,100):
    data_dict = {}
    date = ''
    first_name = ''
    last_name = ''
    full_name = ''
    prefix = ''
    ssn = ''
    phone = ''
    email = ''
    gender = ''
    streetaddress = ''
    zipCode = ''
    city = ''
    state = ''
    country = ''
    price = ''

    try:
        date = df['DATE'][i]
        if date.strip()!= '':
            data_dict['date'] = date
    except Exception:
        pass
    try:
        prefix = df['NAME_TITLE'][i]
        if prefix.strip()!= '':
            data_dict['prefix'] = prefix
    except Exception:
        pass
    try:
        full_name = df['FULL_NAME'][i]
        if full_name.strip()!='':
            data_dict['fullName'] = full_name
    except Exception:
        pass
    try:
        first_name = df['FIRST_NAME'][i]
        x1 = full_name.split()
        if full_name.strip() != " ":
            first_name = x1[0]
            data_dict['firstName'] = first_name
    except Exception:
        pass
    try:
        x1 = []
        last_name = df['LAST_NAME'][i]
        x1 = full_name.split()
        if full_name.strip() != " ":
            last_name = x1[1]
            data_dict['lastName'] = last_name
    except Exception:
        pass
    try:
        ssn = df['SSN'][i]
        if ssn.strip()!='':
            data_dict['ssn'] = ssn
    except Exception:
        pass
    try:
        phone = df['PHONE'][i]
        if phone.strip()!='':
            data_dict['contactNo.'] = phone
    except Exception:
        pass
    try:
        email = df['EMAIL'][i]
        if email.strip()!='':
            data_dict['email'] = email
    except Exception:
        try:
            temp_email=generate_email(full_name)
            data_dict['email'] = temp_email
        except Exception:
              pass
    try:
        gender = df['GENDER'][i]
        if gender.strip()!='':
            data_dict['gender'] = gender
    except Exception:
        temp= get_gender(first_name)
        data_dict['gender'] = temp
    try:
        streetaddress = df['STREET_ADDRESS'][i]
        if streetaddress.strip()!='':
            data_dict['streetAddress'] = streetaddress
    except Exception:
        pass
    try:
        zipCode = df['ZIP_CODE'][i]
        if len(str(zipCode))>3:
            zip_temp= str(zipCode).rstrip('.0')
            data_dict['zipCode'] = zip_temp
        else:
            try:
                temp = df['CITY'][i]
                zipCode=get_zipcode(temp)
                if zipCode == None:
                    pass
                else:
                    data_dict['zipCode'] = zipCode
            except Exception:
                pass
    except Exception:
        pass
    try:
        city = df['CITY'][i]
        if city.strip()!='':
            data_dict['city'] = city
    except Exception:
        try:
            city=get_city(int(zipCode))
            if city == None:
                pass
            else:
                data_dict['city'] = city
        except Exception:
            pass
    try:
        state = df['STATE'][i]
        if state.strip()!='':
            data_dict['state'] = state
    except Exception:
        try:
            state=get_state(int(zipCode))
            if state == None:
                pass
            else:
                data_dict['state'] = state
        except Exception:
            pass
    try:
        country = df['COUNTRY'][i]
        if country.strip()!='':
            data_dict['country'] = country
    except Exception:
        try:
            country=get_country(state)
            if country == None:
                pass
            else:
                data_dict['country'] = country
        except Exception:
            pass

    try:
        price = df['PRICE'][i]
        if price.strip()!='':
            data_dict['price'] = price
    except Exception:
        pass
    if full_name!=' ':
        data_list.append(data_dict)

with open("data_file_tree.json", "w") as write_file:
    json.dump(data_list, write_file)